# Deep Learning Explained

# Module 4 - Lab - Introduction to Regularization for Deep Neural Nets     



This lesson will introduce you to the principles of regularization required to successfully train deep neural networks. In this lesson you will:

1. Understand the need for regularization of complex machine learning models, particularly deep NNs. 
2. Know how to apply constraint-based regularization using the L1 and L2 norms.
3. Understand and apply the concept of data augmentation. 
4. Know how to apply dropout regularization. 
5. Understand and apply early stopping. 
6. Understand the advantages of various regularization methods and know when how to apply them in combination. 




## 1.0 Why do we need regularization for deep learning?

Deep learning models have a great many parameters (weights) which must be fit. This situation arises from the wide and deep architectures that are required to achieve significant **model capacity** for representing complex functions. The core issue is that over-fit models will simply learn the training data and **over-fit models do not generalize**. Therefore, regularization methods are required in order to prevent over-fitting.

In particular, we can point to three interrelated problems with training deep neural networks:

1. Neural network models have large numbers of parameters (weights). With any finite size data set, there is likely to be a low ratio of cases per parameter or low ratio of cases to features. 
2. As a result of the large numbers of parameters, neural networks are susceptible to noise in the training data. Neural networks are generally considered less robust to noise than shallow machine learning methods. 
3. Presumably as a result of the model complexity, neural networks often return unexpected predictions for data cases outside the training data domain. This property has been referred to as **brittleness**. Brittleness has proven to be a serious problem in some production systems. 

The regularization methods presented here will limit these effects. However, there is no 'silver bullet'! Neural networks are hard to train under the best of circumstances. 

### 1.1 Bias-variance trade-off

To better understand this trade-off let's decompose mean square error for a model as follows:

$$\Delta y = E \big[ Y - \hat{f}(X) \big]$$

Where,     
$Y = $ the label vector.  
$X = $ the feature matrix.   
$\hat{f}(x) = $ the trained model.   

Expanding this relation gives us:

$$\Delta y = \big( E[ \hat{f}(X)] - \hat{f}(X) \big)^2 + E \big[ ( \hat{f}(X) - E[ \hat{f}(X)])^2 \big] + \sigma^2\\
\Delta y = Bias^2 + Variance + Irreducible\ Error$$


Regularization will reduce variance, but increase bias. Regularization parameters must be chosen to minimize $\Delta x$. In many cases, this will prove challenging. 

Notice that the **irreducible error** is the limit of model accuracy. Even if we had a perfect model with no bias or variance, the irreducible error is inherent in the data and problem. 

### 1.2 Demonstration of over-parameterization

Let's try a simple example. We will construct a regression models with different numbers of parameters and therefore different model capacities. 

As a first step, we will create a simple single regression model of some synthetic data. The code in the cell below creates data computed from as a straight line, but with considerable Normally distributed random noise. A plot is then created of the result. Execute this code and examine the resulting plot.  

In [ ]:
%matplotlib inline
import numpy as np
import numpy.random as nr
import matplotlib.pyplot as plt
from numpy.random import normal, seed
import sklearn.linear_model as slm
from sklearn.preprocessing import scale
import sklearn.model_selection as ms
from math import sqrt
import keras
import keras.models as models
import keras.layers as layers
from keras.layers import Dropout, LeakyReLU
from keras import regularizers
from keras.layers.normalization import BatchNormalization
from tensorflow import set_random_seed

seed(34567)
x = np.arange(start = 0.0, stop = 10.0, step = 0.25) 
y = np.add(x, normal(scale = 2.0, size = x.shape[0]))

plt.scatter(x,y)

Notice that these data points fall approximately on a straight line, but with significant deviations. 

Next, you will compute a simple single regression model. This model has an intercept term and a single slope parameter. The code in the cell below splits the data into randomly selected training and testing subsets. Execute this code.

In [ ]:
indx = range(len(x))
seed(9988)
indx = ms.train_test_split(indx, test_size = 20)
x_train = np.ravel(x[indx[0]])
y_train = np.ravel(y[indx[0]])
x_test = np.ravel(x[indx[1]])
y_test = np.ravel(y[indx[1]])

Next, we will use the linear model in `sklearn.linear_model` package to create a single regression model for these data. The code in the cell below does just this, prints the single model coefficient, and plots the result. Execute this code. 


In [ ]:
def plot_reg(x, y_score, y):
    ax = plt.figure(figsize=(6, 6)).gca() # define axis
    
    ## Get the data in plot order
    xyy = sorted(zip(x,y_score,y))
    x = [x for x,_,_ in xyy]
    y_score = [ys for _,ys,_ in xyy]
    y = [y for _,_,y in xyy]

    ## Plot the result
    plt.plot(x, y_score, c = 'red')
    plt.scatter(x, y)
    plt.title('Predicted line with test data')

def reg_model(x, y):
    mod = slm.LinearRegression()
    x_scale = scale(x)  # .reshape(-1, 1)
    mod.fit(x_scale, y)
    print(mod.coef_)
    return mod, x_scale, mod.predict(x_scale)

mod, x_scale, y_hat = reg_model(x_train.reshape(-1, 1), y_train)

plot_reg(x_scale, y_hat, y_train)

Examine these results. Notice that the single coefficient (slope) seems reasonable, given the standardization of the training data. Visually, the fit to the training data also looks reasonable. 

We should also test the fit to some test data. The code in the cell does just this and returns the RMS error. execute this code.

In [ ]:
from math import sqrt
def test_mod(x,y, mod):
    x_scale = scale(x)
    y_score = mod.predict(x_scale)
    plot_reg(x_scale, y_score, y)
    return np.std(y_score - y)

test_mod(x_test.reshape(-1, 1), y_test, mod)

Again, these results look reasonable. The RMSE is relatively small given the significant dispersion in these data. 

Now, try a model with significantly higher capacity. In this case we compute new features for a 9th order polynomial model. Using this new set of features a regression model is trained and a summary displayed. 

In [ ]:
seed(2233)
x_power = np.power(x_train.reshape(-1, 1), range(1,10))
x_scale = scale(x_power)

mod_power = slm.LinearRegression()
mod_power.fit(x_scale, y_train)
y_hat_power = mod_power.predict(x_scale)

plot_reg(x_scale[:,0], y_hat_power, y_train)

print(mod_power.coef_)
print(np.std(y_hat_power - y_train))

Notice the following, indicating the model is quite over-fit. 
- There is a wide range of coefficient values across 7 orders of magnitude. This situation is in contrast to the coefficient of the single regression model which had a reasonable single digit value.
- The graph of the fitted model shows highly complex behavior. In reality, this behavior indicates the model is 'learning the data'.  

Now, we will try to test the model with the held-back test data. The code in the cell below creates the same features and applies the `predict` method to the model using these test features. 

In [ ]:
x_test_scale = scale(x_test.reshape(-1, 1)) # Prescale to prevent numerical overflow. 
x_test_power = np.power(x_test_scale, range(1,10))
x_scale_test = scale(x_test_power)

y_hat_power = mod_power.predict(x_scale_test)

plot_reg(x_scale_test[:,0], y_hat_power, y_test)

print(np.std(y_hat_power - y_test))

This is clearly a terrible fit! The RMSE is enormous and the curve of predicted values bears little resemblance to the test values. Indeed, this is a common problem with over-fit models that the errors grow in very rapidly toward the edges of the training data domain. We can definitely state that this model **does not generalize**. 

## 2.0 l2 regularization

We will now explore one of the mostly widely used regularization methods, often referred to as l2 regularization. 

The same method goes by some other names, as it has been 'invented' several times. In particular, this method is known as, **Tikhonov regularization**, **l2 norm regularization**, **pre-whitening** in engineering, and for linear models **ridge regression**. In all likelihood the method was first developed by the Russian mathematician Andrey Tikhonov in the late 1940's. His work was not widely known in the West since his short book on the subject, [Solution of Ill-Posed Problems](https://www.researchgate.net/publication/44438630_Solutions_of_ill-posed_problems_Andrey_N_Tikhonov_and_Vasiliy_Y_Arsenin), was only published in English in 1977, about 30 years after it had appeared in Russian.

![](img/Tikhonov_board.jpg)
<center> **Figure 2.1   
Commemorative plaque for Andrey Nikolayevich Tikhonov at Moscow State University**


So, what is the basic idea? l2 regularization applies a **penalty** proportional to the **l2** or **Euclidean norm** of the model weights to the loss function. The total loss function then becomes:  

$$J(W) = J_{MLE}(W) + \lambda ||W||^2$$

Where,

$$||W||^2 = \big( w_1^2 + w_2^2 + \ldots + w_n^2 \big)^{\frac{1}{2}} = \Big( \sum_{i=1}^n w_i^2 \Big)^{\frac{1}{2}}$$

We call $||W||^2$ the l2 norm of the weights since we square the power of the weights, sum, and then take the square root, or $\frac{1}{2}$ power. 

You can think of this penalty as constraining the 12 or Euclidean norm of the model weight vector. The value of the hyperparameter $\lambda$ determines how much the norm of the coefficient vector constrains the solution. You can see a view of this geometric interpretation in Figure 2.2 below.  

![](img/L2.jpg)
<center> **Figure 2.2. Geometric view of l2 regularization**

Notice that for a constant value of l2, the values of the model parameters $B1$ and $B2$ are related. For example, if $B1$ is maximized then $B2 \sim 0$, or vice versa. It is important to note that l2 regularization is a **soft constraint**. Coefficients are driven close to, but likely not exactly to, zero.   


### 2.1 Regularization for regression  

Let's go back to the regression example. Recall that the 9th order polynomial regression model was massively over-fit. Can l2 regularization help this situation? We can create a model applying regularization and find out. 

The code in the cell below uses the `Ridge` model from `sklearn.linear_model`. The `Ridge` model has an argument `alpha` which corresponds to the regularization parameter, in the notation we have been using. Execute the code and examine the result. 

In [ ]:
mod_L2 = slm.Ridge(alpha = 100.0)
mod_L2.fit(x_scale, y_train)
y_hat_L2 = mod_L2.predict(x_scale)

print(np.std(y_hat_L2 - y_train))
print(mod_L2.coef_)

plot_reg(x_train, y_hat_L2, y_train)

This model is quite different from the un-regularized one we trained previously. 
- The coefficients all have small values. Some of the coefficients are significantly less than 1. These small coefficients are a direct result of the l2 penalty.
- The fitted curve looks rather reasonable given the noisy data.

Now test the model on the test data. Execute the code in the cell below and examine the results. 

In [ ]:
y_hat_L2 = mod_L2.predict(x_scale_test)

plot_reg(x_scale_test[:,0], y_hat_L2, y_test)

print(np.std(y_hat_L2 - y_test))

This result looks a lot more reasonable. The RMSE is nearly the same as for the single feature regression example. Also, the predicted curve looks reasonable.

In summary, we have seen that l2 regularization significantly improves the result for the 9th order polynomial regression. The coefficients are kept within a reasonable range and the predictions are much more reasonable than the unconstrained model. 

### 2.2 l2 regularization for deep learning models 

So, you may well wonder, how well l2 regularization applies to neural networks? Let's give it a try using the 9th order polynomial data.  

The code in the cell below defines and fits the regression model with a single hidden layer with 128 units. No regularization is applied in this first model. 

In [ ]:
nr.seed(345)
set_random_seed(4455)
nn = models.Sequential()
nn.add(layers.Dense(128, activation = 'relu', input_shape = (9, )))
nn.add(layers.Dense(1))
nn.compile(optimizer = 'rmsprop', loss = 'mse', metrics = ['mae'])
history = nn.fit(x_scale, y_train, 
                  epochs = 30, batch_size = 1,
                  validation_data = (x_scale_test, y_test),
                   verbose = 0)

With the model fit, let's have a look at the loss function vs. training epoch. Execute the code in the cell below and examine the result. 

In [ ]:
def plot_loss(history):
    train_loss = history.history['loss']
    test_loss = history.history['val_loss']
    x = list(range(1, len(test_loss) + 1))
    plt.plot(x, test_loss, color = 'red', label = 'Test loss')
    plt.plot(x, train_loss, label = 'Train loss')
    plt.legend()
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Loss vs. Epoch')
    
plot_loss(history) 

It looks like this model becomes overfit after 3 or 4 training epochs. 

Execute the code in the cell below to compute and plot predictions for the unconstrained model. 

In [ ]:
history = nn.fit(x_scale, y_train, 
                  epochs = 4, batch_size = 1,
                  validation_data = (x_scale_test, y_test),
                   verbose = 0)
predicted = nn.predict(x_scale_test)
plot_reg(x_scale_test[:,0], predicted, y_test)
print(np.std(predicted - y_test))

Both the high RMSE and the odd behavior of the predicted curve indicates that this model does not generalize well at all. Notice in particular, how the predicted curve moves away from the test data values on the right. 

Now, we will try to improve this result by applying l2 norm regularization to the neural network. The code in cell below adds l2 regularization to the model. Execute the code and examine the results.

In [ ]:
nr.seed(45678)
set_random_seed(45546)
nn = models.Sequential()
nn.add(layers.Dense(128, activation = 'relu', input_shape = (9, ),
                        kernel_regularizer=regularizers.l2(2.0)))
nn.add(layers.Dense(1))
nn.compile(optimizer = 'rmsprop', loss = 'mse', metrics = ['mae'])
history = nn.fit(x_scale, y_train, 
                  epochs = 30, batch_size = 1,
                  validation_data = (x_scale_test, y_test),
                  verbose = 0)
plot_loss(history)

This loss function is quite a bit different than for the unconstrained model. It is clear that regularization allows many more training epochs before over-fitting. 

But are the predictions any better? Execute the code in the cell below and find out. 

In [ ]:
history = nn.fit(x_scale, y_train, 
                  epochs = 30, batch_size = 1,
                  validation_data = (x_scale_test, y_test),
                  verbose = 0)
predicted = nn.predict(x_scale_test)
plot_reg(x_scale_test[:,0], predicted, y_test)
print(np.std(predicted - y_test))

The l2 regularization has reduced the RMSE. Just as significantly, the pathological behavior of the predicted values on the right is reduced, but clearly not eliminated. The bias effect is also visible. Notice that the left part of the fitted curve is now shifted upwards. 

****************
**Exercise 1:** You have now tried l2 regularization with one choice of regularization hyperparameter, namely the regularization parameter. Finding a good choice for the regularization parameter can require some trial and error. The objective is to find a value that produces a minimum test error.

In the code cells below, create models as follows: 
1. A regularization parameter of 20.0, using a `numpy.random.seed` of 9456 and `set_random_seed` for the TensorFlow backend of 55566
2. A regularization parameter of 200.0, using a `numpy.random.seed` of 9566 and `set_random_seed` for the TensorFlow backend of 44223. 

Plot the loss history for both models. Make sure you give you models different names. 

In [ ]:
nr.seed(9456)
set_random_seed(55566)


In [ ]:
nr.seed(9566)
set_random_seed(44223)


Next, in the cells below you will create code to compute and plot the predicted values from your model for the test data, along with the error metric. Include the test data values on your plot. 

In [ ]:
history20 = nn20.fit(x_scale, y_train, 
                  epochs = 30, batch_size = 1,
                  validation_data = (x_scale_test, y_test),
                  verbose = 0)
predicted20 = nn20.predict(x_scale_test)
plot_reg(x_scale_test[:,0], predicted20, y_test)
print(np.std(predicted20 - y_test))

In [ ]:
history200 = nn200.fit(x_scale, y_train, 
                  epochs = 30, batch_size = 1,
                  validation_data = (x_scale_test, y_test),
                  verbose = 0)
predicted200 = nn200.predict(x_scale_test)
plot_reg(x_scale_test[:,0], predicted200, y_test)
print(np.std(predicted200 - y_test))

Finally, compare the results for the three models with regularization hyperparameter values of 2.0, 20.0, and 200.0. Notice how the RMSE improves as the hyperparameter increases. Notice also, that the test loss for the highest hyperparameter value decreases most uniformly, indicating less over-fitting of the model. 

## 3.0 l1 regularization

We can also do regularization using other norms. The **l1 regularization** or **Lasso**  method limits the sum of the absolute values of the model coefficients. The l1 norm is sometime know as the **Manhattan norm**, since distance are measured as if you were traveling on a rectangular grid of streets. This is in contrast to the l2 norm that measures distance 'as the crow flies'. 

We can compute the l1 norm of the weights as follows:

$$||W||^1 = \big( |w_1| + |w_2| + \ldots + |w_n| \big) = \Big( \sum_{i=1}^n |w_i| \Big)^1$$

where $|x|$ is the absolute value of $x$. 

Notice that to compute the l1 norm, we raise the sum of the absolute values to the first power.

As with l2 regularization, in l1 regularization  we use a penalty term of the l1 norm of the weights. A penalty multiplier, $\alpha$, determines how much the norm of the coefficient vector constrains values of the weights. The complete loss function then becomes: 

$$J(W) = J_{MLE}(W) + \alpha ||W||^1$$

You can see a view of this geometric interpretation in Figure 3.1 below.  

![](img/L1.jpg)
<center> **Figure 3.1. Geometric view of L1 regularization**

Notice that in Figure 3.1 if $B1 = 0$ then $B2$ has a value at the limit, or vice versa. In other words, using a l1 norm constraint forces some weight values to zero to allow other coefficients to take correct values. In this way, the l1 norm constraint **knocks out** some weights from the model altogether. In contrast to l2 regularization, l1 regularization will drive some coefficients to exactly zero. 

### 3.1 l1 regularization

With these ideas in mind, let's apply l1 norm regularization to the 9th order polynomial regression problem. The code in cell below applies l1 regularized or Lasso regularization to the linear regression problem. Execute this code and examine the results. 

In [ ]:
mod_L1 = slm.Lasso(alpha = 2.0, max_iter=100000)
mod_L1.fit(x_scale, y_train)
y_hat_L1 = mod_L1.predict(x_scale)

print(np.std(y_hat_L1 - y_test))
print(mod_L1.coef_)

plot_reg(x_train, y_hat_L1, y_train)

Notice the following about the results of this l1 regularized regression:
- Many of the coefficients are 0, as expected.
- The fitted curve looks reasonable. 

Now, execute the code in the cell below and examine the prediction results. 

In [ ]:
y_hat_L1 = mod_L1.predict(x_scale_test)

plot_reg(x_scale_test[:,0], y_hat_L1, y_test)

print(np.std(y_hat_L1 - y_test))

The RMSE has been reduced considerably, and is less than for l2 regularization regression. The plot of predicted values looks similar to the single regression model, but with some bias. 


### 3.2 Neural network with l1 regularization

Now, we will try l1 regularization with a neural network. The code in the cell below defines, fits and plots a single layer neural network using l1 regularization. Execute this code and examine the results.

In [ ]:
nn = models.Sequential()
nn.add(layers.Dense(128, activation = 'relu', input_shape = (9, ),
                        kernel_regularizer=regularizers.l1(10.0)))
nn.add(layers.Dense(1))
nn.compile(optimizer = 'rmsprop', loss = 'mse', metrics = ['mae'])
history = nn.fit(x_scale, y_train, 
                  epochs = 100, batch_size = 1,
                  validation_data = (x_scale_test, y_test),
                  verbose = 0)
plot_loss(history)

As a result of the l1 regularization the training loss does not exhibit signs of over-fitting for quite a few epochs. 

Next, excute the code in the cell below to compute and display predicted values from the trained network. 

In [ ]:
history = nn.fit(x_scale, y_train, 
                  epochs = 40, batch_size = 1,
                  validation_data = (x_scale_test, y_test),
                  verbose = 0)
predicted = nn.predict(x_scale_test)
plot_reg(x_scale_test[:,0], predicted, y_test)
print(np.std(predicted - y_test))

These results are a definite improvement. The RMSE is similar to that produced by the l2 regularization neural network. Further, the fitting curve shows similar behavior and bias. This bias is the result of the regularization. 

## 4.0 Early stopping

Early stopping is conceptually simple. Early stopping terminates the training of the neural network model at an epoch before it becomes terribly over-fit. That's it! That is the idea of early stopping.

In fact, we have already been using early stopping as we create and test the foregoing regularized models. The question here is, how do we automate this process? 

### 4.1 Early stopping algorithm

The early stopping algorithm simple. This pseudo code shows the basic loop for early stopping on first epoch with a lower performance metric, which is executed after the first training epoch of the model. 

`Do while TRUE:  
    store current model parameters   
    update model for epoch  
    if(performance_for_epoch < stored_performance_metric)  
        return stored_model  
    else  
        stored_performance_metric = performance_for_epoch   
        store_model = model  
`  



### 4.2 How does early stopping work?

Early stopping terminates model learning before over-fitting occurs. But how can we interpret this action in terms of the loss function $J(W)_{MLE}$? Figure 4.1 below provides some insight.   

![](img/EarlyStopping.JPG)    
<center>**Figure 4.1 Effect of early stopping on $J(W)_{MLE}$**</center>

On the left side of the diagram you can see contours of the weight norm. On the right are contours  Early stopping terminates training at some model weight norm $||W||^2$. Ideally this is at the point where the training of $J(W)_{MLE}$ starts to over-fit. Thus, we can think of early stopping as analogous to l2 norm regularization where we write the loss function as:

$$argmin_W J(W) = J(W)_{MLE} + \alpha ||W||^2$$

where,

$\alpha = $ a regularization parameter controlling the stopping point. 

### 4.3 Early stopping example

Manually applying early stopping is both computationally inefficient and rather tedious. Fortunately, Keras has a build in capability that allows automation. 

To implement this early stopping we need to define 2 Keras **callbacks**. Two such callbacks are required:
1. The first callback, **EarlyStopping**, is for the early stopping method.
2. The second call back **checkpoints** or saves the current model. 

These callbacks are defined in the form of a **callbacks list**. 

Notice that the model defined includes l2 regularization. Thus, this model should replicate the performance observed with manual early stopping. To see how this works, examine and then execute the code in the following cell.

In [ ]:
## First define and compile a model. 
nn = models.Sequential()
nn.add(layers.Dense(128, activation = 'relu', input_shape = (9, ),
                        kernel_regularizer=regularizers.l2(1.0)))
nn.add(layers.Dense(1))

nn.compile(optimizer = 'RMSprop', loss = 'mse', metrics = ['mae'])

## Define the callback list
filepath = 'my_model_file.hdf5' # define where the model is saved
callbacks_list = [
    keras.callbacks.EarlyStopping(
        monitor = 'val_loss', # Use loss to monitor the model
        patience = 1 # Stop after one step with lower accuracy
    ),
    keras.callbacks.ModelCheckpoint(
        filepath = filepath, # file where the checkpoint is saved
        monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
        save_best_only = True # Only save model if it is the best
    )
]

## Now fit the model
nr.seed(5566)
set_random_seed(6767)
history = nn.fit(x_scale, y_train, 
                  epochs = 40, batch_size = 1,
                  validation_data = (x_scale_test, y_test),
                  callbacks = callbacks_list,  # Call backs argument here
                  verbose = 0)

## Visualize the outcome
plot_loss(history)

You can see the behavior of the loss with training epoch is behaving as with l2 regularization alone. Notice that the training has been automatically terminated at the point the loss function is at its optimum. 

Let's also have a look at the accuracy vs. epoch. Execute the code in the cell below and examine the result. 

In [ ]:
def plot_accuracy(history):
    train_acc = history.history['mean_absolute_error']
    test_acc = history.history['val_mean_absolute_error']
    x = list(range(1, len(test_acc) + 1))
    plt.plot(x, test_acc, color = 'red', label = 'Test error rate')
    plt.plot(x, train_acc, label = 'Train error rate')
    plt.legend()
    plt.xlabel('Epoch')
    plt.ylabel('Error rate')
    plt.title('Error Rate vs. Epoch')  
    
plot_accuracy(history)  

The curve of test accuracy is consistent with the test loss.

The code in the cell below retrieves the best model (by our stopping criteria) from storage, computes predictions and displays the result. Execute this code and examine the results. 

In [ ]:
best_model = keras.models.load_model(filepath)
predictions = best_model.predict(x_scale_test)
plot_reg(x_scale_test[:,0], predictions, y_test)
print(np.std(predictions - y_test))

As expected, these results are similar, but a bit worse, than those obtained while manually stopping the training of the l2 regularized neural network.  

## 5.0 Dropout regularization

All of the regularization methods we have discussed so far, originated long before the current deep neural network era. We will now look at the **dropout regularization** method. Of all widely used regularization methods, dropout is one of the few specifically developed for neural networks. The seminal paper, [Dropout: A Simple Way to Prevent Neural Networks from
Overfitting by Srivastava et. al](http://www.cs.toronto.edu/~rsalakhu/papers/srivastava14a.pdf), 2014, is quite readable and provides a lot more detail than is presented here.

We have already seen how l1 norm regularization knocks out some model weights. The dropout method regularizes neural networks by creating an **ensemble** of networks with some fraction $p \lt 1.0$ of the hidden units removed. Ensemble methods are know to be strong regularizers and produce superior results by combining the learning of multiple **weak learners**. 

The dropout method is somewhat different from other ensemble methods, such as bagging. This reweighting scheme has several advantages:
- The model weights for the resulting networks are reweighted by the probabilities that they are sampled in the ensemble. 
- The memory required to train the model is simply $O(n)$, where n is the number of weights. A bagged model requires  $O(M*n)$, where $M$ is the number of models in the ensemble.
- When making predictions in production only one model is used. Whereas, the predictions for each model in the bag must be computed for bagging. 

To understand this method, let's recall the basic model for the output of a lth layer in a fully connected network:

$$z^{(l+1)}_i = w^{(l+1)}_i \cdot h^{(l)} + b^{(l+1)}_i\\
h^{(l+1)}_i = \sigma(z^{(l+1)}_i)$$

where:

$\sigma = $ the activation function. 

Now, we need to sample the hidden units with probability $p$, in which case we can write:

$$r^{(l)}_i \sim Bernoulli(p)\\
\tilde{h}^{(l)}_i = r^{(l)}_i * y^{(l)}\\
z^{(l+1)}_i = w^{(l+1)}_i \cdot \tilde{h}^{(l)}_i + b^{(l+1)}_i\\
h^{(l+1)}_i = \sigma(z^{(l+1)}_i)$$

where:

$r^{(l)}_i =$dropout vector with values $\{0,1\}$.

To get a feel for what this means in practice examine Figure 5.1. This figure shows a fully connected network with 4 hidden units and a dropout probability $p = 0.5$. 

![](img/DropoutExample.JPG)
![](img/DropoutExample2.JPG)

<center>**Figure 5.1   
Possible dropouts for a simple fully connected network with p = 0.5**</center>

Examine Figure 5.1 and notice the following:

- There are 6 ways to achieve dropout with exactly 1/2 the units as shown. 
- No units might dropout with probability $p^4$. 
- A single unit might drop out with probability $p^3 (1-p)$. 
- All units might drop out with probability $(1-p)^4$. This case is not admissible so should not be sampled. 

In fact there are $n^2$ possible dropout patterns for a hidden layer with n units. This scaling quickly leads to a problem. For any realistic size network, it is not possible to fully sample all of the possibilities. Instead, we need to use some kind of approximation with a reasonable number of samples. 

Ideally, we want a model that gives us the posterior probability of $y$, the output, given $x$ the input which we can write $p(y\ |\ x)$. If we had infinite computing resources we could Monte Carlo sample this distribution for our neural network. This ideal reference neural network is known as **Bayesian network**. Clearly, for large scale networks it is not possible to compute this result.   

We have to settle for a sampled result. We reweight by the probability that a sample is created. Continuing with the notation we used before we can write:

$$p(y\ |\ x) \sim \sum_r p(r) p(y\ |\ x, r)$$

where,

$r = $ the Bernoulli sampled mask vector. 

Given enough samples the approximation above will converge to the desired probability distribution. However, in practice it has been found that the **geometric mean** of the ensemble converges faster. 

### 5.1 Computing a neural network with dropout regularization

With a bit of theory in mind, we will now apply dropout regularization to training a neural network. The code in the cell below defines a neural network with a dropout layer with $p =0.5$. The rest of this network is identical to other networks we have been working with. Execute this code and examine the result. 

In [ ]:
## First define and compile a model with a dropout layer. 
nn = models.Sequential()
nn.add(layers.Dense(128, activation = 'relu', input_shape = (9, )))
nn.add(Dropout(rate = 0.5)) # Use 50% dropout on this model
nn.add(layers.Dense(1))
nn.compile(optimizer = 'rmsprop', loss = 'mse', metrics = ['mae'])

## Now fit the model
nr.seed(1144)
set_random_seed(6723)
history = nn.fit(x_scale, y_train, 
                  epochs = 40, batch_size = 1,
                  validation_data = (x_scale_test, y_test),
                  callbacks = callbacks_list,  # Call backs argument here
                  verbose = 0)

## Visualize the outcome
plot_loss(history)

The familiar loss plot looks a bit different here. Notice the kinks in the training loss curve. This is likely a result of the dropout sampling. 

Execute the code in the cell below, and examine the accuracy vs. epoch curves. 

In [ ]:
plot_accuracy(history)

The behavior of the training accuracy curve has a similar appearance to the loss curve in terms of the jagged appearance. 

Execute the code in the cell below examine the prediction results for this model. 

In [ ]:
best_model = keras.models.load_model(filepath)
predictions = best_model.predict(x_scale_test)
plot_reg(x_scale_test[:,0], predictions, y_test)
print(np.std(predictions - y_test))

These results appear similar to those obtained with other regularization methods for neural networks on this problem, particularly, early stopping. While the dropout method is an effective regularizer it is no 'silver bullet'. 

## 6.0 Batch Normalization

It is often the case that the distribution of output values of some hidden layers changes . The result is that propagated gradients can become near zero, significantly slowing convergence in many cases. We will discuss this **vanishing gradient problem** in another lesson.  

In 2015, [Sergey and Szegedy](https://arxiv.org/pdf/1502.03167.pdf) introduced a solution to this problem with their paper **Batch Normalization: Accelerating Deep Network Training by Reducing Internal Covariate Shift**. The basic idea is simple. A batch normalization layer maintains an exponential moving average estimate of the mean and variance of the outputs of layer. These values are used to normalize the output values of that layer. In other words, the batch normalization layer ensures the distribution of the output values are constant. 

Let's try an example. The simple neural network model defined in the code cell below includes a batch normalization layer. Also notice that to improve convergence the early stopping has been modified to have a patience of 3. Execute this code.      

In [ ]:
## Use patience of 3
callbacks_list = [
    keras.callbacks.EarlyStopping(
        monitor = 'val_loss', # Use loss to monitor the model
        patience = 3 # Stop after three steps with lower accuracy
    ),
    keras.callbacks.ModelCheckpoint(
        filepath = filepath, # file where the checkpoint is saved
        monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
        save_best_only = True # Only save model if it is the best
    )
]


## Now, define an NN model using batch normalization. 
## First define and compile a model with a batch normalization layer. 
nn = models.Sequential()
nn.add(layers.Dense(128, input_shape = (9, ), activation = 'relu'))
nn.add(BatchNormalization(momentum = 0.99))
nn.add(layers.Dense(1))
## Define the optimizer and compile
optm = keras.optimizers.rmsprop(lr=1.0)
nn.compile(optimizer = optm, loss = 'mse', metrics = ['mae'])

## Now fit the model
nr.seed(345)
set_random_seed(4532)
history = nn.fit(x_scale, y_train, 
                  epochs = 100, batch_size = 1,
                  validation_data = (x_scale_test, y_test),
                  callbacks = callbacks_list,  # Call backs argument here
                  verbose = 0)

## Visualize the outcome
plot_loss(history)

The loss decreases rapidly and then remains in a narrow range thereafter. It appears that convergence is quite rapid.

How does the accuracy evolve with the training episodes? Execute the code in the cell below to display the result.   

In [ ]:
plot_accuracy(history)

This accuracy curve is rather unusual. It seems to reflect the simple regularization being used. 

Finally, execute the code in the cell below to evaluate the predictions made with this model. 

In [ ]:
best_model = keras.models.load_model(filepath)
predictions = best_model.predict(x_scale_test)
plot_reg(x_scale_test[:,0], predictions, y_test)
print(np.std(predictions - y_test))

The fit to the test data look fairly good. 

## 7.0 Using multiple regularization methods

**Exercise 2:** In many cases more than one regularization method is applied. We have already applied early stopping with other regularization methods. In this exercise you will create a neural network work using four regularization methods at once:
- l2 regularization
- l1 regularization
- Dropout
- Early stopping 

In the cell below create code for a neural network using the above regularization methods. Your code should include the following:

1. Set a `numpy.random` seed of 242244 and a `set_random_seed` for the TensorFlow backend of 4356.
2. Define a call back list with `EarlyStopping` with monitor set to `val_loss` and patience set to 4, and the `ModelCheckpoint` with monitor set to `val_loss`
3. A fully connected layer with 128 units and ReLU activation. Include l1 and l2 regulaization using the `l1_l2` function with the regularization parameter set to 50.0.
4. A dropout layer with regularization parameter set to 0.5.
5. Fit your model for 120 epochs, with batch size of 10, using the already defined callback list

Fit you neural network model, saving the results to a history object. 

In [ ]:
nr.seed(242244)
set_random_seed(4346)



In the cell below create and execute the code to plot the loss history for both training and test. 

In [ ]:
## Visualize the outcome
plot_loss(history)

In the cell below create and execute the code to plot the accuracy history for both training and test. 

In [ ]:
plot_accuracy(history)

Next, in the cells below you will create code to compute and plot the predicted values from your model for the test data, along with the error metric. Include the test data values on your plot. 

In [ ]:
best_model = keras.models.load_model(filepath)
predictions = best_model.predict(x_scale_test)
plot_reg(x_scale_test[:,0], predictions, y_test)
print(np.std(predictions - y_test))

How do these results compare to using single regularization methods. Has there been any improvement in accuracy? What about the bias in the fitted curve which is quite noticeable when some of the single regularization methods are used? 